In [3]:
import sqlite3

In [4]:
DB_PATH = "Books.db"
connection = sqlite3.connect(DB_PATH)

In [5]:
cursor = connection.cursor()


cursor.execute('''
    CREATE TABLE  IF NOT EXISTS Book(
        id INTEGER PRIMARY KEY,
        author TEXT,
        title TEXT,
        publish_year TEXT
    );
''')


connection.commit()

In [6]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Reader(
        id INTEGER PRIMARY KEY,
        name TEXT
    );
''')


connection.commit()

In [7]:
cursor.execute('''
    CREATE TABLE IF NOT EXISTS Record(
        book_id INT NOT NULL,
        reader_id INT NOT NULL,
        taking_date TEXT,
        returning_date TEXT,
        FOREIGN KEY (book_id) REFERENCES Book (id),
        FOREIGN KEY (reader_id) REFERENCES Reader (id)
    );
''')


connection.commit()

In [8]:

cursor.execute('''
    INSERT INTO Reader VALUES
        (1, 'Reader 1'),
        (2, 'Reader 2'),
        (3, 'Reader 3'),
        (4, 'Reader 1');
''')

connection.commit()

IntegrityError: UNIQUE constraint failed: Reader.id

In [9]:
cursor.execute('''
    INSERT INTO Book VALUES
        (1, 'author 1', 'title 1', '2002'),
        (2, 'author 2', 'title 2', '2002'),
        (3, 'author 3', 'title 3', '2003'),
        (4, 'author 1', 'title 4', '2002'),
        (5, 'author 1', 'title 1', '2004'),
        (6, 'author 4', 'title 5', '2005'),
        (7, 'author 5', 'title 6', '2022');
''')

connection.commit()

IntegrityError: UNIQUE constraint failed: Book.id

In [22]:
cursor = connection.cursor()
cursor.execute('''
    DELETE FROM Record
''')

cursor.execute('''
    INSERT INTO Record VALUES
        (1, 1, '04.03.2010', '04.04.2010'),
        (1, 2, '04.03.2011', '11.22.2011'),
        (2, 5, '04.03.2012', '05.23.2012'),
        (1, 4, '04.03.2010', '04.14.2010'),
        (3, 1, '05.03.2010', '04.03.2011');
''')

connection.commit()

In [23]:
cursor = connection.cursor()

cursor.execute("SELECT * FROM Book WHERE id = 1")
print(cursor.fetchall(), '\n')

cursor.execute("SELECT * FROM Reader WHERE id = 1")
print(cursor.fetchall(), '\n')

cursor.execute("SELECT * FROM Record WHERE book_id = 2")
print(cursor.fetchall(), '\n')

[(1, 'author 1', 'title 1', '2002')] 

[(1, 'Reader 1')] 

[(2, 5, '04.03.2012', '05.23.2012')] 



Вывод списка книг

In [53]:
def list_of_books():
    cursor = connection.cursor()

    book_index = list(cursor.execute("SELECT id FROM Book"))
    for i in book_index:
        cursor.execute("SELECT title FROM Book WHERE id={0}".format(i[0]))
        print(cursor.fetchall()[0][0])

list_of_books()

title 1
title 2
title 3
title 4
title 1
title 5
title 6
